In [1]:
with open("API_KEY.txt") as f:
    CREDENTIALS = {"API_KEY" : f.readline().strip().split("=")[-1]}

print(CREDENTIALS)

{'API_KEY': '5805b27b'}


In [3]:
import requests
import json

BASE_URL = f"http://www.omdbapi.com/?apikey={CREDENTIALS['API_KEY']}&"

def fetch(query):
    COMPOSITE_URL = f"{BASE_URL}{query}"
    print(COMPOSITE_URL)
    with requests.get(COMPOSITE_URL) as r:
        film_dict = json.loads(r.content)
        if r.status_code < 400:
            return film_dict
        Response.raise_for_status()

search_title = input("TITLE>>>")
query = "+".join(search_title.split())
film = fetch(f"t={query}")
film

TITLE>>>the batman
http://www.omdbapi.com/?apikey=5805b27b&t=the+batman


{'Title': 'The Batman',
 'Year': '2004–2008',
 'Rated': 'TV-Y7',
 'Released': '11 Sep 2004',
 'Runtime': '30 min',
 'Genre': 'Animation, Action, Adventure, Crime, Family, Mystery, Sci-Fi, Thriller',
 'Director': 'N/A',
 'Writer': 'N/A',
 'Actors': 'Rino Romano, Alastair Duncan',
 'Plot': 'Billionaire Bruce Wayne fights crime and evil as the mysterious Batman.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': '7 wins & 13 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BZjE2MzA0YTgtNjQ3MS00MjMxLWE2MzMtOGYzMTZjZjFiYTI0XkEyXkFqcGdeQXVyOTgwMzk1MTA@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}],
 'Metascore': 'N/A',
 'imdbRating': '7.3',
 'imdbVotes': '9,290',
 'imdbID': 'tt0398417',
 'Type': 'series',
 'totalSeasons': '5',
 'Response': 'True'}

In [39]:
def prune():
    rel_dict = {}

    relevant_fields = ["Title", "Runtime", "Genre", "Director", "Ratings", "imdbRating", "BoxOffice"]
    scores = ["imdbRating", "Internet Movie Database", "Rotten Tomatoes", "Metacritic"]

    for key_ in relevant_fields:
        if key_ in film.keys():
            rel_dict[key_] = film[key_]

    try:
        rel_dict["Genre"] = rel_dict["Genre"].split(", ")
        rel_dict["Director"] = rel_dict["Director"].split(", ")
    except:
        pass
    
    sources = {}
    try:
        sources = {d["Source"] : d["Value"] for d in rel_dict["Ratings"]}
    except:
        pass
    
    for k, v in sources.items():
        rel_dict[k] = v

    for score in scores:
        split_char = "/"
        if score == "Rotten Tomatoes":
            split_char = "%"
        value = None
        try:
            value = float(rel_dict[score].split(split_char)[0]) or None
        except:
            pass
        finally:
            rel_dict[score] = value
            value = None
        
    rel_dict.pop("Ratings", None)
    return rel_dict

rel_dict = prune()

In [42]:
FILENAME = "data.json"

def write_to_file(FILENAME, rel_dict):
    json_string = json.dumps(rel_dict, separators=(",",":"))
    with open(FILENAME, "a") as infile:
        infile.write(json_string+"\n")

write_to_file(FILENAME, rel_dict)

import pandas as pd
df = pd.read_json("data.json", lines=True)
df

,Title,Runtime,Genre,Director,imdbRating,BoxOffice,Internet Movie Database,Rotten Tomatoes,Metacritic
0,For Better or Worse,7 min,"[Short, Drama]",[Rodney Owen],NaN,N/A,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Papa,N/A,[N/A],[N/A],NaN,NaN,NaN,NaN,NaN
7,Private Movies 47: ABC of Fucking,N/A,[Adult],[Antonio Adamo],NaN,N/A,NaN,NaN,NaN
8,Stoker,99 min,"[Drama, Thriller]",[Chan-wook Park],6.8,"$1,700,000",6.8,69.0,58.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
from random import randint

for i in range(100):
    id_ = f"tt{randint(1, 1877830)}"
    film = fetch(f"i={id_}&type=movie")
    rel_film = prune()
    write_to_file(FILENAME, rel_film)

http://www.omdbapi.com/?apikey=5805b27b&i=tt1713116&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt1273354&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt348258&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt750975&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt181778&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt33156&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt856330&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt1002666&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt1077451&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt255897&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt259431&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt1101872&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt831528&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt1747631&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt341042&type=movie
http://www.omdbapi.com/?apikey=5805b27b&i=tt1485900&type=movie
ht